In [ ]:
# --- 1. Import thư viện ---
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Nếu chưa cài, hãy chạy: pip install xgboost lightgbm
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# --- 2. Đọc dữ liệu ---
df_hist = pd.read_csv('techco_financials_with_variables.csv')
df_forecast = pd.read_csv('techco_forecast_input.csv')

# --- 3. Tiền xử lý ---
df_hist['Date'] = pd.to_datetime(df_hist['Date'])
df_forecast['Date'] = pd.to_datetime(df_forecast['Date'])

features = ['Labor Cost', 'Production Cost', 'Other Costs', 'CPI (%)', 'Material Cost (%)']
target = 'Profit'  # Có thể đổi thành 'Revenue (M USD)' nếu muốn

X = df_hist[features]
y = df_hist[target]

# --- 4. Tách tập train/test ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 5. Khởi tạo các mô hình ---
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=200, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=200, random_state=42, verbosity=0),
    "LightGBM": LGBMRegressor(n_estimators=200, random_state=42)
}

results = []

# --- 6. Huấn luyện và đánh giá ---
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results.append({'Model': name, 'MAE': mae, 'R2': r2})

results_df = pd.DataFrame(results)
print("So sánh các mô hình:")
print(results_df)

# --- 7. Chọn mô hình tốt nhất ---
best_model_name = results_df.sort_values('R2', ascending=False).iloc[0]['Model']
print(f"\nMô hình tốt nhất: {best_model_name}")

# Huấn luyện lại mô hình tốt nhất trên toàn bộ dữ liệu
best_model = models[best_model_name]
best_model.fit(X, y)

# --- 8. Dự báo cho dữ liệu mới ---
X_forecast = df_forecast[['Labor Cost', 'Production Cost Adjusted', 'Other Costs', 'CPI (%)', 'Material Cost (%)']]
X_forecast = X_forecast.rename(columns={'Production Cost Adjusted': 'Production Cost'})
df_forecast['Profit Forecast'] = best_model.predict(X_forecast)

print("\nKết quả dự báo lợi nhuận:")
print(df_forecast[['Date', 'Profit Forecast']])

# --- 9. Xuất kết quả ra file ---
df_forecast.to_csv('profit_forecast_output.csv', index=False)
print("\nĐã lưu kết quả dự báo vào profit_forecast_output.csv")